## Pulp Scheduling


In [62]:
from dataclasses import dataclass
from copy import deepcopy
import math
from typing import Tuple, List

### Classes


In [72]:
@dataclass
class Topic:
    name: str
    hours: float
    
@dataclass
class Course:
    name: str
    topics: List[Topic]
    final_day: int
    relative_proportion: float

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

### Functions


In [104]:
def round_to_nearest_half(number: float) -> float:
    return round(number * 2) / 2


def validate_courses(courses: Tuple[Course], hours_per_day: List[float]) -> None:
    total_fraction = 0
    n_days = len(hours_per_day)
    for course in courses:
        total_fraction += course.relative_proportion
        if course.final_day > n_days:
            raise ValueError(
                f"The final day of {course.name} ({course.final_day}) "
                f"is outside the number of days ({n_days})."
            )


    if total_fraction != 1:
        raise ValueError(f"Relative proportion did not sum to 1 ({total_fraction})")



def perform_course_scale(course: Course, factor: float) -> Course:
    topics = course.topics
    scaled_topics = []
    for topic in topics:
        scaled_hours = topic.hours * factor
        rounded_hours = round_to_nearest_half(scaled_hours)
        scaled_topic = Topic(topic.name, rounded_hours)
        scaled_topics.append(scaled_topic)

    scaled_course = Course(
        course.name, scaled_topics, course.final_day, course.relative_proportion
    )
    return scaled_course

def obtain_topic_hours(topic: Topic) -> float:
    return topic.hours

def enforce_hours(course: Course, required_hours: float) -> Course:
    enforced_course = deepcopy(course)
    topics = enforced_course.topics
    # Ordering topics from longest time to least
    ordered_topics = sorted(topics, key=obtain_topic_hours, reverse=True)
    current_hours = enforced_course.total_hours()

    i = 0
    while not (math.isclose(current_hours, required_hours)):
        index = i % len(topics)
        if current_hours < required_hours:
            ordered_topics[index].hours += 0.5  # Increase by 30 minutes if less than required
        else:
            ordered_topics[index].hours -= 0.5  # Decrease by 30 minutes if greater than required
        current_hours = enforced_course.total_hours()
        i += 1

    return enforced_course

def schedule_hour_split(courses: List[Course], total_study_hours: float) -> list[float]:
    hours_split = []
    for i in range(len(courses) - 1):
        course = courses[i]
        hours_to_study = round_to_nearest_half(course.relative_proportion * total_study_hours)
        hours_split.append(hours_to_study)

    # Adding the remaining time
    # Round to nearest 0.5
    hours_to_study = round_to_nearest_half(total_study_hours - sum(hours_split))
    hours_split.append(hours_to_study)

    return hours_split


def scale_courses(courses: List[Course], hours_per_day: List[int]) -> List[Course]:
    validate_courses(courses, hours_per_day)
    # Determining the number of hours required for each course for the given schedule
    total_study_hours = sum(hours_per_day)
    hours_split = schedule_hour_split(courses, total_study_hours)

    # Performing the rescale
    rescaled_courses = []
    for course, hours_to_study in zip(courses, hours_split):
        rescaling_factor = hours_to_study / course.total_hours()
        rescaled_course = perform_course_scale(course, rescaling_factor)
        rescaled_courses.append(rescaled_course)

    # Enforcing that the number of hours in each course matches the scaled hours
    enforced_courses = []
    for rescaled_course, hours_to_study in zip(rescaled_courses, hours_split):
        enforced_course = enforce_hours(rescaled_course, hours_to_study)
        enforced_courses.append(enforced_course)

    return enforced_courses

def obtain_final_day(course: Course) -> int:
    return course.final_day

def is_viable(courses: List[Course], hours_per_day: List[int]) -> bool:
    # Order courses by final day
    ordered_courses = sorted(courses, key=obtain_final_day)

    # Creating a list of the available number of hours for each course
    available_hours_list = []
    start_index = 0
    for course in ordered_courses:
        end_index = course.final_day + 1
        available_hours = sum(hours_per_day[start_index:end_index])
        available_hours_list.append(available_hours)
        start_index = end_index

    # Checking if the study schedule viable within the time constraints
    remainder = 0
    for i, course in enumerate(ordered_courses):
        available_hours_list[i] += remainder
        remainder = available_hours_list[i] - course.total_hours()
        if remainder < 0:
            # Study schedule is not viable
            return False
        
    # Study schedule is viable
    return True


def allocate_topics(
    courses: List[Course], hours_per_day: List[int]
) -> List[List[Topic]]:
    # Ensure immutability of original variables
    courses = deepcopy(courses)
    hours_per_day = hours_per_day.copy()

    # Order courses by their final day from closest to furthest away from start
    ordered_courses = sorted(courses, key=obtain_final_day)

    # Combine all topics into a single list
    topics = []
    for course in ordered_courses:
        for topic in course.topics:
            topics.append(deepcopy(topic))

    # Distribute topics to each day of the schedule
    schedule = [[] for _ in range(len(hours_per_day))]
    day = 0
    for topic in topics:
        while topic.hours > 0 and day < len(hours_per_day):
            if hours_per_day[day] > 0:
                hours_to_add = min(topic.hours, hours_per_day[day])
                schedule[day].append(Topic(topic.name, hours_to_add))
                topic.hours -= hours_to_add
                hours_per_day[day] -= hours_to_add
            if hours_per_day[day] == 0:
                day += 1

    return schedule

### Testing


In [10]:
# Inputs
# 1)

# Maths
linear_algebra = Topic("Linear Algebra", 1)
geometry = Topic("Geometry", 2)
differential_equations = Topic("Differential Equations", 4)
maths = Course("Maths", [linear_algebra, geometry, differential_equations], 1, 0.2)

# Physics
o_and_w = Topic("Oscillations and Waves", 3)
mechanics = Topic("Mechanics", 3)
quantum_physics = Topic("Quantum Physics", 4)
electromagnetism = Topic("Electromagnetism", 10)

physics = Course(
    "Physics", [o_and_w, mechanics, quantum_physics, electromagnetism], 6, 0.8
)

# Creating the schedule
courses = [maths, physics]
hours_per_day = [2, 4, 3, 1, 3, 0.5]  # Total = 13.5


scaled_courses = scale_courses(courses, hours_per_day)
print(is_viable(scaled_courses, hours_per_day))
daily_schedule = allocate_topics(scaled_courses, hours_per_day)

for day in daily_schedule:
    print(day)

NameError: name 'scale_courses' is not defined

In [4]:
from dataclasses import dataclass
from typing import List, Dict
from datetime import datetime
from copy import deepcopy


@dataclass
class Topic:
    id: int
    title: str
    hours: float

@dataclass
class Plan:
    id: int
    title: str
    final_day: int
    fraction: float
    required_hours: float
    topics: List[Topic]

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

# Utilities
def n_days_difference(date: str, reference_date: str) -> int:
    """Determine the number of days between an input date and a reference date."""
    date_format = "%Y-%m-%d"

    date = datetime.strptime(date, date_format)
    reference_date = datetime.strptime(reference_date, date_format)

    date_difference = date - reference_date
    return date_difference.days


# Validate that the plans provided are suitable for forming a schedule
def validate_plans(plans: List[Plan], n_days: int) -> None:
    """
    Validates that the list of plans fractions sum to 1 and that none of the plan'
    final days are outside the number of days.
    """
    total_fraction = 0
    for plan in plans:
        total_fraction += float(plan.fraction)
        if plan.final_day > n_days - 1:
            raise IndexError(
                f"The final day of {plan.title} ({plan.final_day}) "
                f"is outside the number of days ({n_days})."
            )

    if total_fraction != 1:
        raise ValueError(f"Relative proportion does not sum to 1 ({total_fraction})")


def allocate_required_hours(plans: List[Plan], total_hours: float) -> None:
    """Assigned the hours required for each plan to match the total hours scheduled."""
    unallocated_hours = total_hours
    for i in range(len(plans) - 1):
        plan = plans[i]
        required_hours = round(plan.fraction * total_hours)
        plan.required_hours = required_hours
        unallocated_hours -= required_hours

    # Adding the remaining unallocated hours
    plan = plans[-1]
    plan.required_hours = unallocated_hours


def obtain_final_day(plan: Plan) -> int:
    return plan.final_day


def are_plan_hours_viable(plans: List[Plan], hours_list: List[int]) -> Dict:
    """
    Checks whether a list of plans with required hours are viable within the
    constraints provided by the study hours list.
    """

    # Order plans by final day
    ordered_plans: List[Plan] = sorted(plans, key=obtain_final_day)

    # Creating a list of the available number of hours for each course
    available_hours_list = []
    start_index = 0
    for plan in ordered_plans:
        end_index = plan.final_day + 1
        available_hours = sum(hours_list[start_index:end_index])
        available_hours_list.append(available_hours)
        start_index = end_index

    # Checking if the study schedule viable within the time constraints
    remainder = 0
    for i, plan in enumerate(ordered_plans):
        available_hours_list[i] += remainder
        remainder = available_hours_list[i] - plan.required_hours
        if remainder < 0:
            # Study schedule is not viable
            return {"is_viable": False, "id": plan.id}

    # Study schedule is viable
    return {"is_viable": True, "id": None}



def format_viability_result(viability_result: Dict, plans_data: List[Dict]) -> None:
    """
    Formats the viability result object to include the title and exam date of the
    plan."""
    for plan_details in plans_data:
        if plan_details["id"] == viability_result["id"]:
            viability_result["title"] = plan_details["title"]
            viability_result["exam_date"] = plan_details["exam_date"]


# Generating the study schedule based on the user input data
def allocate_topics(plans: List[Plan], hours_list: List[int]) -> List[List[Topic]]:
    """
    Allocates the topics in each plan to each day adhering to the constraint
    of the number of hours in a day.
    """
    # Ensure immutability of original variables
    plans = deepcopy(plans)
    hours_list = hours_list.copy()

    # Order plans by their final day from closest to furthest away from start
    ordered_plans = sorted(plans, key=obtain_final_day)

    # Combine all topics into a single list
    topics = []
    for course in ordered_plans:
        for topic in course.topics:
            topics.append(deepcopy(topic))

    # Distribute topics to each day of the schedule
    schedule = [[] for _ in range(len(hours_list))]
    day = 0
    for topic in topics:
        while topic.hours > 0 and day < len(hours_list):
            if hours_list[day] > 0:
                hours_to_add = min(topic.hours, hours_list[day])
                schedule[day].append(Topic(topic.id, topic.title, hours_to_add))
                topic.hours -= hours_to_add
                hours_list[day] -= hours_to_add
            if hours_list[day] == 0:
                day += 1

    return schedule


In [71]:
hours_list = [10,10]

# Chemistry
topic1 = Topic(1, "Chemistry Topic 1", )
chemistry = Plan(10, "Chemistry", 0, 0.5, required_hours=10)

[10.0, 5.0, 5.0]


['1.0', '3.0', '4.0', '2.0']
